In [ ]:
import os
import torch
from torchvision import datasets, transforms
import torchvision.models as models
import torch.nn as nn
from PIL import Image


In [ ]:
# Define the transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to load images from a directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            image = Image.open(img_path)
            image = transform(image)  # Apply transformation
            images.append(image)
    return torch.stack(images)  # Stack images into a tensor

# Load your dataset
real_images_path = '/content/drive/MyDrive/ColabNotebooks/datasets/trainA'
selfie_images_path = '/content/drive/MyDrive/ColabNotebooks/datasets/trainB'

real_images = load_images_from_folder(real_images_path)
selfie_images = load_images_from_folder(selfie_images_path)

print("Real Images Shape: ", real_images.shape)
print("Selfie Images Shape: ", selfie_images.shape)

Real Images Shape:  torch.Size([1244, 3, 224, 224])
Selfie Images Shape:  torch.Size([1190, 3, 224, 224])


In [ ]:
# Load VGG19 model
vgg = models.vgg19(pretrained=True).features
vgg.eval()  # Set to evaluation mode

# Move VGG to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg = vgg.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Function to extract features
def extract_features(images, model):
    features_list = []

    with torch.no_grad():
        for img in images:
            img = img.unsqueeze(0).to(device)  # Add batch dimension and move to device
            features = model(img)  # Extract features
            features_list.append(features.cpu())  # Store the features on CPU

    return features_list

# Extract features from both datasets
real_features = extract_features(real_images, vgg)
selfie_features = extract_features(selfie_images, vgg)

# Convert to tensors for further processing
real_features_tensor = torch.cat(real_features, dim=0)
selfie_features_tensor = torch.cat(selfie_features, dim=0)

print("Real Features Shape: ", real_features_tensor.shape)
print("Selfie Features Shape: ", selfie_features_tensor.shape)

Real Features Shape:  torch.Size([1244, 512, 7, 7])
Selfie Features Shape:  torch.Size([1190, 512, 7, 7])


In [ ]:

# Define the PerceptualLoss class
class PerceptualLoss(nn.Module):
    def __init__(self, feature_extractor):
        super(PerceptualLoss, self).__init__()
        self.feature_extractor = feature_extractor
        self.criterion = nn.MSELoss()  # Mean Squared Error Loss for perceptual loss

    def forward(self, real_image, generated_image):
        # Extract features for both real and generated images
        real_features = self.feature_extractor(real_image)
        generated_features = self.feature_extractor(generated_image)

        # Calculate perceptual loss as the MSE between the features
        loss = self.criterion(real_features, generated_features)
        return loss


In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 3 * 224 * 224),  # Output layer for 224x224 RGB images
            nn.Tanh()  # Use Tanh to scale output between -1 and 1
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 3, 224, 224)  # Reshape to image dimensions
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3 * 224 * 224, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1),  # Output layer for binary classification
            nn.Sigmoid()  # Sigmoid to output probabilities
        )

    def forward(self, x):
        x = x.view(-1, 3 * 224 * 224)  # Flatten the images
        return self.model(x)


In [ ]:
# Hyperparameters
latent_dim = 100
num_epochs = 100  # Set your number of epochs
batch_size = 32  # Set your batch size

# Initialize the models
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# Loss and Optimizers
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999))


In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class CustomImageDataset(Dataset):
    def __init__(self, real_images, selfie_images):
        self.real_images = real_images
        self.selfie_images = selfie_images

    def __len__(self):
        return max(len(self.real_images), len(self.selfie_images))

    def __getitem__(self, idx):
        real_image = self.real_images[idx % len(self.real_images)]  # Loop through real images
        selfie_image = self.selfie_images[idx % len(self.selfie_images)]  # Loop through selfie images
        return real_image, selfie_image

# Create the dataset
dataset = CustomImageDataset(real_images, selfie_images)

# Load your dataset using DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for real_images, selfie_images in dataloader:
        real_images = real_images.to(device)
        selfie_images = selfie_images.to(device)  # Use the selfie images if needed

        # Create labels
        real_labels = torch.full((real_images.size(0), 1), 0.9).to(device)  # Real label smoothing
        fake_labels = torch.full((real_images.size(0), 1), 0.1).to(device)  # Fake label smoothing


        # Train Discriminator
        optimizer_D.zero_grad()
        output = discriminator(real_images)
        loss_D_real = criterion(output, real_labels)

        noise = torch.randn(real_images.size(0), latent_dim).to(device)  # Latent space noise
        fake_images = generator(noise)
        output = discriminator(fake_images.detach())
        loss_D_fake = criterion(output, fake_labels)

        loss_D = loss_D_real + loss_D_fake
        loss_D.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        output = discriminator(fake_images)
        loss_G = criterion(output, real_labels)  # Want to fool the discriminator
        loss_G.backward()
        optimizer_G.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss D: {loss_D.item()}, Loss G: {loss_G.item()}')


Epoch [1/100], Loss D: 1.058280110359192, Loss G: 0.7090227603912354
Epoch [2/100], Loss D: 1.2238678932189941, Loss G: 0.7122827768325806
Epoch [3/100], Loss D: 2.9160094261169434, Loss G: 0.9068707227706909
Epoch [4/100], Loss D: 1.1950573921203613, Loss G: 0.6591204404830933
Epoch [5/100], Loss D: 1.0259792804718018, Loss G: 0.7705346941947937
Epoch [6/100], Loss D: 1.087127447128296, Loss G: 0.7386795282363892
Epoch [7/100], Loss D: 1.0384941101074219, Loss G: 0.7679590582847595
Epoch [8/100], Loss D: 1.030881404876709, Loss G: 0.7973692417144775
Epoch [9/100], Loss D: 1.0670127868652344, Loss G: 0.7571843266487122
Epoch [10/100], Loss D: 1.0315558910369873, Loss G: 0.8014633059501648
Epoch [11/100], Loss D: 1.200903058052063, Loss G: 0.7627974152565002
Epoch [12/100], Loss D: 1.0928423404693604, Loss G: 0.8190175890922546
Epoch [13/100], Loss D: 1.1571204662322998, Loss G: 0.7599086165428162
Epoch [14/100], Loss D: 1.0971794128417969, Loss G: 0.9622009992599487
Epoch [15/100], Los

In [ ]:
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp generator.pth /content/drive/MyDrive/


In [ ]:
latent_dim = 100  # Example value, should match the one used during training

# Initialize your generator model with the correct latent dimension
generator = Generator(latent_dim=latent_dim)
generator.load_state_dict(torch.load('/content/drive/MyDrive/generator.pth'))
generator.eval()  # Set to evaluation mode


<ipython-input-34-9633e3cd9023>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load('/content/drive/MyDrive/generator.pth'))


Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=150528, bias=True)
    (7): Tanh()
  )
)

In [ ]:
!pip install streamlit
!pip install pyngrok


In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import streamlit as st
from PIL import Image
from torchvision import transforms

# Define the Generator class
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 3 * 224 * 224),  # Output layer for 224x224 RGB images
            nn.Tanh()  # Use Tanh to scale output between -1 and 1
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 3, 224, 224)  # Reshape to image dimensions
        return x

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the generator
latent_dim = 100
generator = Generator(latent_dim=latent_dim).to(device)

# Load the generator state_dict
generator.load_state_dict(torch.load('/content/drive/MyDrive/generator.pth', map_location=device))
generator.eval()

# Define the transformation (if needed for input image)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to generate a selfie
def generate_selfie():
    with torch.no_grad():
        latent_vector = torch.randn(1, latent_dim).to(device)  # Generate random latent vector
        generated_selfie = generator(latent_vector)
    generated_selfie = generated_selfie.squeeze(0).cpu().numpy()  # Remove batch dimension
    generated_selfie = (generated_selfie * 0.5 + 0.5) * 255  # Rescale to [0, 255]
    generated_selfie = generated_selfie.astype(np.uint8)
    generated_selfie_image = Image.fromarray(generated_selfie.transpose(1, 2, 0))  # Convert to PIL Image
    return generated_selfie_image

# Streamlit app
st.title("Selfie Generator")
st.write("Upload an image to generate a selfie!")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Load the uploaded image
    user_image = Image.open(uploaded_file)
    st.image(user_image, caption='Uploaded Image', use_column_width=True)

    # Generate and display the selfie
    if st.button("Generate Selfie"):
        generated_selfie = generate_selfie()
        st.image(generated_selfie, caption='Generated Selfie', use_column_width=True)


<ipython-input-36-7bc6927151e1>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load('/content/drive/MyDrive/generator.pth', map_location=d

In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.12.93:8501



In [ ]:
import os
import torch
import numpy as np
import streamlit as st
from PIL import Image
from torchvision import transforms
from your_gan_model import Generator  # Import your generator class here

# Set the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained generator model
latent_dim = 100  # Match with your training setup
generator = Generator(latent_dim).to(device)  # Move model to device
generator.load_state_dict(torch.load('generator.pth', map_location=device))
generator.eval()

# Define the transformation (only if necessary for user image)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to generate a selfie
def generate_selfie():
    # Generate the selfie using the generator
    with torch.no_grad():
        latent_vector = torch.randn(1, latent_dim).to(device)  # Generate random latent vector
        generated_selfie = generator(latent_vector)

    # Convert tensor to image
    generated_selfie = generated_selfie.squeeze(0).cpu().numpy()  # Remove batch dimension
    generated_selfie = (generated_selfie * 0.5 + 0.5) * 255  # Rescale to [0, 255]
    generated_selfie = generated_selfie.astype(np.uint8)
    generated_selfie_image = Image.fromarray(generated_selfie.transpose(1, 2, 0))  # Convert to PIL Image

    return generated_selfie_image

# Streamlit app
st.title("Selfie Generator")
st.write("Click the button below to generate a random selfie!")

# Generate and display the selfie when button is clicked
if st.button("Generate Selfie"):
    generated_selfie = generate_selfie()
    st.image(generated_selfie, caption='Generated Selfie', use_column_width=True)


ModuleNotFoundError: No module named 'your_gan_model'

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
%cd pytorch-CycleGAN-and-pix2pix
!pip install -r requirements.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!python train.py --dataroot /content/drive/MyDrive/ColabNotebooks/datasets --name cycle_gan_experiment --model cycle_gan --display_id -1


In [ ]:
import os

# List contents of the datasets directory
dataset_path = '/content/drive/MyDrive/ColabNotebooks/datasets'
print("Contents of datasets directory:", os.listdir(dataset_path))

# List contents of trainA directory
trainA_path = os.path.join(dataset_path, 'NonSelfie')
print("Contents of trainA directory:", os.listdir(trainA_path))

# List contents of trainB directory
trainB_path = os.path.join(dataset_path, 'Selfie')
print("Contents of trainB directory:", os.listdir(trainB_path))


Contents of datasets directory: ['Selfie', 'NonSelfie']
Contents of trainA directory: ['NonSelfie10221.jpg', 'NonSelfie1022.jpg', 'NonSelfie10224.jpg', 'NonSelfie10209.jpg', 'NonSelfie10223.jpg', 'NonSelfie10220.jpg', 'NonSelfie10215.jpg', 'NonSelfie10210.jpg', 'NonSelfie10214.jpg', 'NonSelfie10225.jpg', 'NonSelfie10240.jpg', 'NonSelfie10236.jpg', 'NonSelfie10239.jpg', 'NonSelfie10232.jpg', 'NonSelfie10235.jpg', 'NonSelfie10228.jpg', 'NonSelfie10231.jpg', 'NonSelfie10226.jpg', 'NonSelfie10233.jpg', 'NonSelfie10227.jpg', 'NonSelfie10237.jpg', 'NonSelfie10238.jpg', 'NonSelfie10229.jpg', 'NonSelfie10230.jpg', 'NonSelfie10234.jpg', 'NonSelfie1024.jpg', 'NonSelfie1023.jpg', 'NonSelfie10258.jpg', 'NonSelfie10247.jpg', 'NonSelfie10255.jpg', 'NonSelfie10256.jpg', 'NonSelfie10252.jpg', 'NonSelfie10253.jpg', 'NonSelfie10245.jpg', 'NonSelfie10244.jpg', 'NonSelfie10243.jpg', 'NonSelfie10242.jpg', 'NonSelfie10254.jpg', 'NonSelfie10241.jpg', 'NonSelfie1025.jpg', 'NonSelfie10248.jpg', 'NonSelfie10251